# Summarize Assignment of On-board Survey Demand
This notebook uses the Acceptance Criteria infrastructure to summarize the results of the assignment of the on-board survey demand to the TM2 networks. 

In [1]:
from tm2py.acceptance.canonical import Canonical
from tm2py.acceptance.simulated import Simulated
from tm2py.acceptance.observed import Observed
from tm2py.acceptance.acceptance import Acceptance

import pandas as pd
import geopandas as gpd

Inro not found. Skipping inro setup.


### Remote I/O

In [2]:
scenario_config_file = "../examples/scenario_config.toml"
observed_config_file = "../examples/observed_data.toml"
model_config_file = "../examples/model_config.toml"
canonical_crosswalk_file = "../examples/canonical_crosswalk.toml"

input_previous_summary_filename = "on-board-assignment-summary.csv"

output_folder_root = "/Users/wsp/Documents/GitHub/tm2py/examples/temp_on_board_assign/"
output_filename = "on-board-assignment-summary.csv"

## Parameters

In [3]:
append_or_create = "append" 
iteration_description = "Iter 1 with Forced Transit Boarding"
iteration_number = 2

In [4]:
c = Canonical(canonical_crosswalk_file, on_board_assign_summary=True)

In [5]:
s = Simulated(canonical=c, scenario_file=scenario_config_file, model_file=model_config_file, on_board_assign_summary=True)
s.reduce_on_board_assignment_boardings(time_period_list=["am"])

In [6]:
s.simulated_boardings_df.head()

,daily_line_name,tm2_mode,line_mode,operator,technology,fare_system,total_boarding,total_hour_cap,time_period,line_name
0,10_573_198,12,b,Emery Go-Round,Local bus,18,14.162698,33.750000,daily,N/A -- Daily Record
1,10_573_199,12,b,Emery Go-Round,Local bus,18,88.317225,207.692308,daily,N/A -- Daily Record
2,10_573_200,12,b,Emery Go-Round,Local bus,18,70.058158,207.692308,daily,N/A -- Daily Record
3,10_573_201,12,b,Emery Go-Round,Local bus,18,19.940998,33.750000,daily,N/A -- Daily Record
4,10_573_202,12,b,Emery Go-Round,Local bus,18,109.503650,192.857143,daily,N/A -- Daily Record


In [7]:
o = Observed(canonical=c, observed_file=observed_config_file, on_board_assign_summary=True)
o.reduce_on_board_survey(read_file_from_disk=True)

In [8]:
a = Acceptance(canonical=c, simulated=s, observed=o, output_file_root=output_folder_root)  
compare_df = a.compare_transit_boardings()
rail_access_gdf = a.make_rail_access_comparisons()

In [9]:
compare_df.head()

,tm2_mode,line_mode,operator,technology,time_period,total_boarding,survey_route,survey_boardings,florida_threshold,daily_line_name,...,line_name,survey_tech,survey_operator,survey_agency,standard_route_id,canonical_operator,standard_route_short_name,standard_route_long_name,standard_line_name,standard_headsign
0,14.0,b,Caltrain,Local Bus,am,36.603022,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,14.0,b,Caltrain,Local Bus,daily,36.603022,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,120.0,h,BART,Heavy Rail,am,153992.549229,BART,134161.945802,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,120.0,h,BART,Heavy Rail,daily,153992.549229,BART,431650.949580,0.25,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,130.0,r,Caltrain,Commuter Rail,am,16053.877656,Caltrain,20978.587956,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
rail_access_gdf.head()

,dimension_02_value,observed_outcome,boarding,boarding_name,boarding_standard_node_id,simulated_outcome,criteria_number,criteria_name,acceptance_threshold,dimension_01_name,dimension_02_name,dimension_01_value,geometry
0,Bike,280.336567,NaN,NaN,NaN,0.0,19,Percent error in share of transit boardings th...,MTC's assessment of reasonableness,Boarding Station,Access Mode,BART Ashby,POINT (nan nan)
1,Kiss and Ride,304.002339,NaN,NaN,NaN,0.0,19,Percent error in share of transit boardings th...,MTC's assessment of reasonableness,Boarding Station,Access Mode,BART Ashby,POINT (nan nan)
2,Park and Ride,666.676503,NaN,NaN,NaN,0.0,19,Percent error in share of transit boardings th...,MTC's assessment of reasonableness,Boarding Station,Access Mode,BART Ashby,POINT (nan nan)
3,Walk,1620.882756,NaN,NaN,NaN,0.0,19,Percent error in share of transit boardings th...,MTC's assessment of reasonableness,Boarding Station,Access Mode,BART Ashby,POINT (nan nan)
4,Bike,357.398508,NaN,NaN,NaN,0.0,19,Percent error in share of transit boardings th...,MTC's assessment of reasonableness,Boarding Station,Access Mode,BART Balboa Park,POINT (nan nan)


## Make Output

In [11]:
compare_df["iteration_description"] = iteration_description
compare_df["iteration_number"] = iteration_number

if append_or_create == "create":
    compare_df.to_csv(output_folder_root + output_filename, index=False)
else:
    previous_df = pd.read_csv(output_folder_root + input_previous_summary_filename, dtype=compare_df.dtypes.to_dict())
    pd.concat([previous_df, compare_df], ignore_index=True).to_csv(output_folder_root + output_filename, index=False)

In [12]:
gdf = rail_access_gdf[["dimension_01_value", "dimension_02_value", "observed_outcome", "simulated_outcome", "geometry"]].copy()
gdf = gdf.rename(columns={"dimension_01_value": "station_name", "dimension_02_value": "access_mode"})
gdf.to_file(output_folder_root + output_pnr_shapefilename, driver="GeoJSON")
